In [1]:
import requests
import json
import pandas as pd
import time

# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running flask from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

# Running with gunicorn (heroku, digitalocean, etc):
# Procfile >
# web: gunicorn wsgi --log-file -
# wsgi.py >
# from api import app as application
#
#if __name__ == "__main__":
#    application.run()

# https://devcenter.heroku.com/articles/getting-started-with-python
# https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-gunicorn-and-nginx-on-ubuntu-14-04

In [2]:
# Set up heroku database > 
# heroku addons:create heroku-postgresql:hobby-dev
# Postgres db now up
# Look at config vars (should have DATABASE_URL) >
# heroku config
# Connect to remote psql > 
# heroku pg:psql
#


In [100]:
# baseurl = 'https://vast-depths-89393.herokuapp.com/'
baseurl = 'http://127.0.0.1:5000/'

In [103]:
url = baseurl +'createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [104]:
response.json()

{'signingKey_hex': '67a91d7aa6f75246cef00a5aeb1307d61db5e9aab0c978c81905524b0e0a3a6a',
 'traderId': '8',
 'verifyKey_hex': '8a9ad5da39e17cad54e65da1ae0baa847c2d5be8a3a9fa8f0e72a5a6fe21869d'}

In [105]:
url = baseurl + 'createMarket'

mkId = 6
# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

In [106]:
content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": mkId, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [107]:
pd.DataFrame(response.json(), index=[0])


,checks,marketBranchId,marketMax,marketMin,marketRootId,traderId
0,True,1,1,0,6,8


In [108]:
# Create a trade
url = baseurl+'createTrade'

content_maketrade = {"signingKey_hex": sk, "traderId": tId, "verifyKey_hex": vk,
  "marketRootId": mkId, "marketBranchId": 1, "price": 0.5678, "quantity":1}

In [109]:
# Post trade
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [110]:
# View json response
pd.DataFrame(response.json(), index=[0])

,checks,marketBranchId,marketRootId,price,quantity,traderId
0,"(True, True)",1,6,0.5678,1,8


In [111]:
# Create a matched trade
url = baseurl+'createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [112]:
response.json()

{'checks': '(True, True)',
 'marketBranchId': 1,
 'marketRootId': 6,
 'price': 0.5678,
 'quantity': -1,
 'traderId': '8'}

In [113]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 5.456560850143433 seconds.
matched trade: 1 in 5.555504083633423 seconds.
matched trade: 2 in 5.49550986289978 seconds.
matched trade: 3 in 5.871792793273926 seconds.
matched trade: 4 in 5.627130746841431 seconds.
matched trade: 5 in 5.439238786697388 seconds.
matched trade: 6 in 5.818356990814209 seconds.
matched trade: 7 in 5.883602142333984 seconds.
matched trade: 8 in 5.7089080810546875 seconds.
matched trade: 9 in 5.732460021972656 seconds.


In [114]:
url = baseurl+'createMarket'

for i in range(4):
    t = time.time()

    content_makemarket = {"signingKey_hex": sk,
      "traderId": int(tId), "verifyKey_hex": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

failed to create market 1
failed to create market 2
failed to create market 3
failed to create market 4


In [115]:
response.json()

{'checks': 'False',
 'marketBranchId': 1,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 4,
 'traderId': 8}

In [116]:
url = baseurl+'viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)



In [117]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,marketMin,marketMax
0,1,1,0,1.0
1,2,1,0,0.7
2,3,1,0,0.7
3,4,1,0,0.7
4,6,1,0,1.0


In [118]:
url = baseurl+'viewOrderBook'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [119]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,price,quantity,traderId
0,1,1,0.5510,1,4
1,1,1,0.5510,-1,4
10,1,1,0.5510,-1,5
100,1,1,0.5510,1,5
101,1,1,0.5510,1,5
102,1,1,0.5510,-1,5
103,1,1,0.5510,-1,5
104,1,1,0.5510,-1,5
105,1,1,0.4400,1,5
106,1,1,0.5510,-1,6


In [94]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = 1
    content_maketrade['price'] = 0.44
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 1.738020896911621 seconds.
matched trade: 1 in 1.5786552429199219 seconds.
matched trade: 2 in 1.6579430103302002 seconds.
matched trade: 3 in 1.5954749584197998 seconds.
matched trade: 4 in 1.6175951957702637 seconds.
matched trade: 5 in 1.6032161712646484 seconds.
matched trade: 6 in 1.6124258041381836 seconds.
matched trade: 7 in 1.6094579696655273 seconds.
matched trade: 8 in 1.6454930305480957 seconds.
matched trade: 9 in 1.693124771118164 seconds.


In [120]:
url = baseurl+'viewOpenTrades'

content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [121]:
pd.read_json(response.json())

,marketRootId,marketBranchId,price,quantity,traderId
0,1,1,0.440,10,3
1,1,1,0.440,10,4
2,1,1,0.440,10,5
3,1,1,0.440,10,6
4,1,1,0.551,-1,6


In [122]:
url = baseurl+'viewMatchedTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [123]:
pd.read_json(response.json())

,marketRootId,marketBranchId,price,quantity,traderId
0,1,1,0.5510,0,3
1,1,1,0.5510,0,4
2,1,1,0.5510,-1,5
3,1,1,0.5510,1,6
4,6,1,0.5678,0,8


In [99]:
tId

'6'